
'KPI'+'Time Spent'=> 'RAW_KPI'생성 
'Referring Domain','Marketing Channel','URL','Product'=> 'RAW_Detail'생성 
'KPI'의 visit컬럼으로 나라별 순위 => 'RAW_Country'

세개의 시트를 US와 이외의 나라별로 정리를 해야한다.
'RAW_KPI', 'RAW_Detail', 'RAW_Country'를 만들어보자.


-> 먼저 어도비에서 추출한 데이터 그대로를 필요한 모양으로 정제해야한다.

In [1]:
# max row, columns 셋팅
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
import numpy as np
from pandas import DataFrame, Series
import os
import datetime
import re


In [2]:
######################mapping 표 불러오기################

sheet=['Country_map','Channel_map','Product_map']

map=Series()
for i in range(0,len(sheet)):
    map[sheet[i]]= pd.read_excel('C:/Users/cnxk/Desktop/dashboard_prac/Map.xlsx',

                              sheet_name = sheet[i], 

                              header = 1, 

                              thousands = ','  )   
    map[sheet[i]]=map[sheet[i]].dropna(axis=1,how='all')#첫째열은 전체 빈행->삭제해주기

C:\Users\cnxk\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


In [3]:
######################어도비 데이터 그대로 불러오기####################


sheet=['KPI','Referring Domain','Marketing Channel','URL','Product','Time Spent']

scom=Series()
for i in range(0,len(sheet)):
    scom[sheet[i]]= pd.read_excel('C:/Users/cnxk/Desktop/dashboard_prac/Scom_Global_june.xlsx',

                              sheet_name = sheet[i], 

                              header = 1, 

                              thousands = ','  )   
    
##dimension열 같은 필요없는 컬럼삭제
    col_not_dimension = []
    for j in scom[sheet[i]].columns:
        if bool(re.match('^Dimension',j)) == False:
            col_not_dimension.append(j)
    scom[sheet[i]]=scom[sheet[i]][col_not_dimension]
    
##자이제 Item컬럼의 빈행, 토탈행, low-traffic 행을 지워야한다.    
    scom[sheet[i]]=scom[sheet[i]].dropna(axis=1,how='all')#첫째열은 전체 빈열->삭제해주기
    scom[sheet[i]]=scom[sheet[i]].dropna(how='any')#NA값들어간 행 지우기
    


    
#필요없는 행들 지우기, 빈줄은 위에서 다지웠고
#KPI -> item=total
#'Referring Domain'->item.1=total
#'Marketing Channel'->item.1=total
#'URL'->item.1=total,(Low-traffic)
#'Product'->item.1=total
#'Time Spent'->item.2=Total

scom['KPI']=scom['KPI'].loc[ (scom['KPI']['Item']!='Total'),]
scom['Referring Domain']=scom['Referring Domain'].loc[ (scom['Referring Domain']['Item.1']!='Total'),]
scom['Marketing Channel']=scom['Marketing Channel'].loc[ (scom['Marketing Channel']['Item.1']!='Total'),]
scom['URL']=scom['URL'].loc[ (scom['URL']['Item.1']!='Total')
                                                      &(scom['URL']['Item.1']!=r'(Low-traffic)'),]
scom['Product']=scom['Product'].loc[ (scom['Product']['Item.1']!='Total'),]
scom['Time Spent']=scom['Time Spent'].loc[ (scom['Time Spent']['Item.2']!='Total'),]


##############사이트코드담긴 열, 인덱스로 빼주기
scom['KPI']=scom['KPI'].set_index('Item',drop=True)
scom['Time Spent']=scom['Time Spent'].reset_index(drop=True).set_index('Item.1',drop=True)



C:\Users\cnxk\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


여기서 time spent 처리는 까다롭다 
합계 : (V-B) 
합계 : (V_B)*T 
합계 : avg. Time Spent를 만들어 줘야한다

먼저 scom['Time Spent']에 (V-B),(V_B)*T, avg. Time Spent 컬럼을 만들어주자

In [4]:
a=scom['Time Spent']['[MST] Memory_41 Sites + [MST] Memory - Visit']
b=scom['Time Spent']['[MST] Memory_41 Sites + [MST] Memory - Bounce']
c=scom['Time Spent']['[MST] Memory_41 Sites + Time Spent per Visit (seconds)']
sub=[x-y for x,y in zip (a,b)]
subT=[x*y for x,y in zip (sub,c)]


scom['Time Spent']['(V-B)']=sub
scom['Time Spent']['(V-B)*T']=subT

#이제 나라별 평균체류시간 구해야한다. 피벗

avg_duration= scom['Time Spent'].pivot_table(values=['(V-B)','(V-B)*T'],
                                                    index=scom['Time Spent'].index,
                                                    aggfunc='sum')

avg_duration=avg_duration[avg_duration['(V-B)']!=0] #혹시모를 분모의 0값들은 아예 행에서 빼준다

avg_duration['Spent Time']=[y/x for x,y in zip (avg_duration['(V-B)'],avg_duration['(V-B)*T'])]

#time spent랑 kpi가 정리 끝,
#두개 붙이면 RAW_KPI 완성~! scom['KPI']에 위의 avg_duration 붙여서 정렬시키고 key값 생성하면 완성

scom['KPI']['Time Spent']=avg_duration['Spent Time']
scom['KPI'].columns=['Visit','Visit - PC','Visit - Mobile','Visitor','Page Views',
                 'Entry','Bounce','SSD','PSSD','Memory Card','USB','PFS','PCD','PF','PD','Time Spent']
scom['KPI']=scom['KPI'][['Visit','Visit - PC','Visit - Mobile','Visitor','Page Views',
                 'Entry','Bounce','Time Spent','SSD','PSSD','Memory Card','USB','PFS','PCD','PF','PD']]
scom['KPI']
#US와 global 빼고, 'RAW_KPI'를 위한 필요 정렬 완료

,Visit,Visit - PC,Visit - Mobile,Visitor,Page Views,Entry,Bounce,Time Spent,SSD,PSSD,Memory Card,USB,PFS,PCD,PF,PD
Item,,,,,,,,,,,,,,,,
ru,76453,40072,36381,67197,124597,67134,22397,69.814070,43006,4976,20260,10512,1200,10284,43245,40929
de,54359,33607,20752,49255,89188,37523,14674,46.740267,37357,8454,5338,3741,1624,10817,20960,33266
in,52327,16368,35959,47402,93106,35757,12605,61.664966,16745,17256,18394,0,3644,11307,24348,29653
cn,38841,31660,7181,34630,100278,17538,4211,54.122468,28138,4664,3930,2325,4099,25187,16162,21708
uk,29537,19520,10017,25726,55448,20692,7288,57.725817,15082,6426,6834,2014,259,6994,7700,23605
be,26457,11178,15279,20657,35187,25474,20091,139.236051,1840,25135,255,125,123,25552,1509,1450
tw,24527,7411,17116,20966,40814,22006,10017,55.166488,14389,5454,4305,609,656,8151,3670,18128
ca,23773,12015,11758,21917,38473,19591,7728,50.725674,8708,13482,1269,9555,1125,7241,2594,18840
es,23477,14684,8793,20727,36976,19680,5087,72.402169,17119,3691,3557,566,765,1,5194,20736


이로써 US이외의 데이터는 필요 컬럼과 행만 추출해냈다. 

'KPI'+'Time Spent'=> 'RAW_KPI'생성 
'Referring Domain','Marketing Channel','URL','Product'=> 'RAW_Detail'생성 
'KPI'의 visit컬럼으로 나라별 순위 => 'RAW_Country'

In [5]:
#대시보드에 들어갈 세 데이터프레임 생성

RAW_KPI=DataFrame(columns=['Site_key','Site','Month','Visit','Visit - PC','Visit - Mobile','Visitor','Page Views',
                 'Entry','Bounce','Time Spent','SSD','PSSD','Memory Card','USB','PFS','PCD','PF','PD'])
RAW_Detail=DataFrame(columns=['Key Value2','Key Value1','Country','Month','Report','Rank', 'Item', 'Value'])

RAW_Country=DataFrame(columns=['Key Value2 (Month & Country)','Key Value1 (Month & Rank)','Month','Rank',
                               'Country','Visit'])


In [14]:
#똑같이 US의 데이터를 정제해보자
#모양이 쫌 다르니 달리 처리해준것들 유의
###############US 같은거 반복
#맨아래 time spent만 컬럼갯수달라 처리
sheet=['KPI','Referring Domain','Marketing Channel','URL','Product','Time Spent']

scom_us=Series()
for i in range(0,len(sheet)):
    scom_us[sheet[i]]= pd.read_excel('C:/Users/cnxk/Desktop/dashboard_prac/Scom_US_june.xlsx',

                              sheet_name = sheet[i], 

                              header = 1  )   
    
##dimension열 같은 필요없는 컬럼삭제
    col_not_dimension = []
    for j in scom_us[sheet[i]].columns:
        if bool(re.match('^Dimension',j)) == False:
            col_not_dimension.append(j)
    scom_us[sheet[i]]=scom_us[sheet[i]][col_not_dimension]
    
##자이제 Item컬럼의 빈행, 토탈행, low-traffic 행을 지워야한다.    
    scom_us[sheet[i]]=scom_us[sheet[i]].dropna(axis=1,how='all')#첫째열은 전체 빈행->삭제해주기
    scom_us[sheet[i]]=scom_us[sheet[i]].dropna(how='any')#NA값들어간 행 지우기
    
    
#필요없는 행들 지우기, 빈줄은 위에서 다지웠고
#KPI -> item=total
#'Referring Domain'->item.1=total
#'Marketing Channel'->item.1=total
#'URL'->item.1=total,(Low-traffic)
#'Product'->item.1=total
#'Time Spent'->item.1=Total

scom_us['KPI']=scom_us['KPI'].loc[ (scom_us['KPI']['Item']!='Total'),]
scom_us['Referring Domain']=scom_us['Referring Domain'].loc[ (scom_us['Referring Domain']['Item.1']!='Total'),]
scom_us['Marketing Channel']=scom_us['Marketing Channel'].loc[ (scom_us['Marketing Channel']['Item.1']!='Total'),]
scom_us['URL']=scom_us['URL'].loc[ (scom_us['URL']['Item.1']!='Total')
                                                      &(scom_us['URL']['Item.1']!=r'Low_Traffic'),]
scom_us['Product']=scom_us['Product'].loc[ (scom_us['Product']['Item.1']!='Total'),]
scom_us['Time Spent']=scom_us['Time Spent'].loc[ (scom_us['Time Spent']['Item.1']!='Total'),]
scom_us['Time Spent']=scom_us['Time Spent'].loc[ (scom_us['Time Spent']['[US] Memory - Visit']!=0),]



#미국 채널 매핑해줘야하고, 상품 모양 바꿔줘야한다- 
#채널정리(None=>Other, Banner=>Display)->아래서 다른나라랑 묶어서 통째루 다해쥼 미국제품이름(GA=>G,BW=>B,/*=> ,DA=>D, MZ들만 B삭제)

scom_us['Product']['Item.1']=scom_us['Product']['Item.1'].str.upper()
scom_us['Product']['Item.1'] = scom_us['Product']['Item.1'].str.replace("GA","G")

scom_us['Product']['Item.1'] = scom_us['Product']['Item.1'].str.replace("BW","B").str.replace(r"/*","").str.replace("DA","D")

product_clean=scom_us['Product']['Item.1'][scom_us['Product']['Item.1'].str.contains('^MZ')]
product_clean=product_clean.str.replace("B","")
product_clean_index=product_clean.index
product_clean_val=list(product_clean)


for i,j in zip(product_clean_index,product_clean_val):
    
    scom_us['Product'].loc[i,'Item.1']=j

###us 상품명 다 고쳤다
scom_us['Product']

#US time spent 고치기
a=scom_us['Time Spent']['[US] Memory - Visit']
b=scom_us['Time Spent']['[US] Memory - Bounce']
c=scom_us['Time Spent']['Time Spent per Visit (seconds)']
sub=[x-y for x,y in zip (a,b)]
subT=[x*y for x,y in zip (sub,c)]

scom_us['Time Spent']['(V-B)']=sub
scom_us['Time Spent']['(V-B)*T']=subT
#이제 나라별 평균체류시간 구해야한다. 피벗

avg_duration= scom_us['Time Spent'].pivot_table(values=['(V-B)','(V-B)*T'],
                                                    index=scom_us['Time Spent'].index,
                                                    aggfunc='sum')

avg_duration=avg_duration[avg_duration['(V-B)']!=0]

avg_duration['Spent Time']=[y/x for x,y in zip (avg_duration['(V-B)'],avg_duration['(V-B)*T'])]
avg_duration['Spent Time']
scom_us['KPI']['Time Spent']=np.array(avg_duration['Spent Time'])
scom_us['KPI']

#원하는 컬럼순서로 맞춰주기
scom_us['KPI']['Time Spent']=avg_duration['Spent Time']
scom_us['KPI']=scom_us['KPI'].drop('Item',axis=1).set_index(Series('us'))

scom_us['KPI'].columns=['Visit','Visit - PC','Visit - Mobile','Visitor','Page Views',
                 'Entry','Bounce','SSD','PSSD','Memory Card','USB','PFS','PCD','PF','PD','Time Spent']
scom_us['KPI']['Time Spent']=np.array(avg_duration['Spent Time'])


scom_us['KPI']=scom_us['KPI'][['Visit','Visit - PC','Visit - Mobile','Visitor','Page Views',
                 'Entry','Bounce','Time Spent','SSD','PSSD','Memory Card','USB','PFS','PCD','PF','PD']]
scom_us['KPI']
#이로써 scom['Time Spent']와 통일해줌

#KPI모양은 다만들었다.
RAW_KPI=pd.concat([scom['KPI'],scom_us['KPI']])
RAW_KPI
#RAW_KPI 기본내용 완성 
#이후에 키값, 나라이름 매핑, 날짜 열 만들어야해

C:\Users\cnxk\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys


ValueError: Length of values does not match length of index


다음으로는 RAW_Detail 만들어보자 목표 모양은 Key Value2 (Month & Country), Key Value1 (Month & Rank), Month, Rank, Country, Visit
먼저 네가지 시트의 값들 정렬하고 순위 매겨서 붙여보자 다음에 Report 컬럼 생성하자

In [247]:
scom_us['Referring Domain']=scom_us['Referring Domain'].set_index(Series(['us' for i in range(len(scom_us['Referring Domain']))]))
Series(['us' for i in range(len(scom_us['Referring Domain']))])

#kpi_detail에 들어갈 네가지 시트 rank, report, Item, value 만들어주기


scom['Referring Domain']=scom['Referring Domain'].reset_index(drop=True).reset_index().set_index('Item',drop=True)
scom['Referring Domain']['index']=scom['Referring Domain']['index']+np.array(1)
scom['Referring Domain'].rename(columns={'index':'Rank'},inplace=True)


scom['Marketing Channel']=scom['Marketing Channel'].reset_index(drop=True).reset_index().set_index('Item',drop=True)
scom['Marketing Channel']['index']=scom['Marketing Channel']['index']+np.array(1)
scom['Marketing Channel'].rename(columns={'index':'Rank'}, inplace=True)


scom['URL']=scom['URL'].reset_index(drop=True).reset_index().set_index('Item',drop=True)
scom['URL']['index']=scom['URL']['index']+np.array(1)
scom['URL'].rename(columns={'index':'Rank'}, inplace=True)


scom['Product']=scom['Product'].reset_index(drop=True).reset_index().set_index('Item',drop=True)
scom['Product']['index']=scom['Product']['index']+np.array(1)
scom['Product'].rename(columns={'index':'Rank'}, inplace=True)



####US 똑같이
#us로 인덱스 값 넣기

scom_us['Referring Domain']=scom_us['Referring Domain'].reset_index(drop=True).reset_index().set_index('Item',drop=True)
scom_us['Referring Domain']['index']=scom_us['Referring Domain']['index']+np.array(1)
scom_us['Referring Domain'].rename(columns={'index':'Rank'},inplace=True)
scom_us['Referring Domain'].rename(columns={'Item.1':'Item'},inplace=True)
scom_us['Referring Domain']=scom_us['Referring Domain'].set_index(Series(['us' for i in range(len(scom_us['Referring Domain']))]))

scom_us['Marketing Channel']=scom_us['Marketing Channel'].reset_index(drop=True).reset_index().set_index('Item',drop=True)
scom_us['Marketing Channel']['index']=scom_us['Marketing Channel']['index']+np.array(1)
scom_us['Marketing Channel'].rename(columns={'index':'Rank'}, inplace=True)
scom_us['Marketing Channel'].rename(columns={'Item.1':'Item'},inplace=True)
scom_us['Marketing Channel']=scom_us['Marketing Channel'].set_index(Series(['us' for i in range(len(scom_us['Marketing Channel']))]))

scom_us['URL']=scom_us['URL'].reset_index(drop=True).reset_index().set_index('Item',drop=True)
scom_us['URL']['index']=scom_us['URL']['index']+np.array(1)
scom_us['URL'].rename(columns={'index':'Rank'}, inplace=True)
scom_us['URL'].rename(columns={'Item.1':'Item'},inplace=True)
scom_us['URL']=scom_us['URL'].set_index(Series(['us' for i in range(len(scom_us['URL']))]))

scom_us['Product']=scom_us['Product'].reset_index(drop=True).reset_index().set_index('Item',drop=True)
scom_us['Product']['index']=scom_us['Product']['index']+np.array(1)
scom_us['Product'].rename(columns={'index':'Rank'}, inplace=True)
scom_us['Product'].rename(columns={'Item.1':'Item'},inplace=True)
scom_us['Product']=scom_us['Product'].set_index(Series(['us' for i in range(len(scom_us['Product']))]))


#컬럼이름 바꾸는 딕셔너리 생성

columns={"[MST] Memory_41 Sites + [MST] Memory - Visit : Channel":"Value",
         "[MST] Memory_41 Sites + [MST] Memory - Visit":"Value"}
#컬럼이름 Value로 바꾸기
#Report 컬럼 생성
#순위 매기기
scom['Referring Domain'].rename(columns=columns, inplace=True)
scom['Referring Domain']['Report']=np.array('Referring Domain')

scom['Marketing Channel'].rename(columns=columns, inplace=True)
scom['Marketing Channel']['Report']=np.array('Marketing Channel')

scom['URL'].rename(columns=columns, inplace=True)
scom['URL']['Report']=np.array('URL')

scom['Product'].rename(columns=columns, inplace=True)
scom['Product']['Report']=np.array('Product')

columns={"[US] Memory - Visit : Channel":"Value",
         "[US] Memory - Visit":"Value"}
    
scom_us['Referring Domain'].rename(columns=columns, inplace=True)
scom_us['Referring Domain']['Report']=np.array('Referring Domain')

scom_us['Marketing Channel'].rename(columns=columns, inplace=True)
scom_us['Marketing Channel']['Report']=np.array('Marketing Channel')

scom_us['URL'].rename(columns=columns, inplace=True)
scom_us['URL']['Report']=np.array('URL')

scom_us['Product'].rename(columns=columns, inplace=True)
scom_us['Product']['Report']=np.array('Product')

columns={"Item":"Item.1"}

scom_us['Referring Domain'].rename(columns=columns, inplace=True)

scom_us['Marketing Channel'].rename(columns=columns, inplace=True)

scom_us['URL'].rename(columns=columns, inplace=True)

scom_us['Product'].rename(columns=columns, inplace=True)


RAW_Detail=pd.concat([scom['Referring Domain'],scom_us['Referring Domain'],scom['Marketing Channel'],scom_us['Marketing Channel'],
                      scom['URL'],scom_us['URL'],scom['Product'],scom_us['Product']])
columns={"Item.1":"Item"}
RAW_Detail.rename(columns=columns, inplace=True)
RAW_Detail=RAW_Detail[['Report','Rank','Item','Value']]

#RAW_Country 만들기

RAW_Country=RAW_KPI['Visit'].sort_values(ascending=False)
RAW_Country=DataFrame(RAW_Country)
RAW_Country=RAW_Country.reset_index().reset_index()
RAW_Country['level_0']=RAW_Country['level_0']+np.array(1)
columns={"level_0":"Rank",
         "index":"Country"}
RAW_Country=RAW_Country.rename(columns=columns).set_index('Country')
RAW_Country

#나라매핑

    
CountryMap=map['Country_map']

CountryMap.set_index('Site',inplace=True)
CountryMap

RAW_KPI=pd.merge(CountryMap, RAW_KPI,left_index=True, right_index=True,how='inner')
#채널값들 바꿔주기
#us 제품명 바꿔주기
RAW_Detail=pd.merge(RAW_Detail, CountryMap,left_index=True, right_index=True, how='inner')

ChannelMap=map['Channel_map'][['Channel Detail_Raw','Channel Detail_Map']].set_index('Channel Detail_Raw')

ChannelMap=ChannelMap.to_dict()
ChannelMap

RAW_Detail['Item']=RAW_Detail['Item'].replace(ChannelMap)
RAW_Detail


RAW_Country=pd.merge(CountryMap, RAW_Country,left_index=True, right_index=True,how='inner').sort_values(by='Visit',ascending=False)

RAW_Country=RAW_Country[['Rank','Country','Visit']]

In [248]:
# 2020/7/9 
# 더해야할것?
# 글로벌 내용 넣기, RAW_KPI 합, RAW_Detail->채널별, 제품명
# RAW Detail 20개씩 자르기

#순위정하는거 이상함


RAW_KPI
GB=RAW_KPI.sum()
GB[0]='Global'
GB
RAW_KPI=RAW_KPI.append(GB,ignore_index=True)
RAW_KPI
# 글로벌 내용 넣기, RAW_KPI 합 생성완료
#이제 RAW_Detail  Item값 따라 합





ValueError: could not convert string to float: 'Global'

In [237]:
#RAW_Detail 글로벌 내용구하기, 상위 20

RAW_Detail_GB= RAW_Detail.pivot_table(values=['Value'],
                                                    index=['Report', 'Item'],
                                                    aggfunc='sum')

RAW_Detail_GB=RAW_Detail_GB.sort_values(by = 'Value',ascending=False).groupby('Report').head(20)


RAW_Detail_GB=RAW_Detail_GB.sort_values(by=['Report', 'Value'],ascending=[True,False])

RAW_Detail_GB=RAW_Detail_GB.reset_index()

RAW_Detail_GB['Country']=np.array('Global')
RAW_Detail_GB

RAW_Detail=RAW_Detail.drop(['Rank'],1)

RAW_Detail=RAW_Detail.sort_values(by = 'Value',ascending=False).groupby(['Report','Country']).head(20)
RAW_Detail=RAW_Detail.sort_values(by=['Country','Report','Value'],ascending=[True,True,False])

RAW_Detail=RAW_Detail.append(RAW_Detail_GB, ignore_index=True)


RAW_Detail

KeyError: "['Rank'] not found in axis"

In [238]:
RAW_Detail_GB

,Report,Item,Value,Country
0,Marketing Channel,Natural Search,806798,Global
1,Marketing Channel,Direct,245946,Global
2,Marketing Channel,Paid Search,191062,Global
3,Marketing Channel,Social Media Campaigns,93778,Global
4,Marketing Channel,Other External Campaign,63360,Global
5,Marketing Channel,PLA,57712,Global
6,Marketing Channel,Session Refresh,48054,Global
7,Marketing Channel,Display,34364,Global
8,Marketing Channel,Other,31324,Global
9,Marketing Channel,Affiliate,23686,Global


In [239]:
writer = pd.ExcelWriter('C:/Users/cnxk/Desktop/dashboard_prac/clean_data_june.xlsx', engine='xlsxwriter')

RAW_Country.to_excel(writer , sheet_name = 'RAW_Country')
RAW_Detail.to_excel(writer , sheet_name = 'RAW_Detail')
RAW_KPI.to_excel(writer , sheet_name = 'RAW_KPI')
writer.save()